In [14]:
import requests
import zipfile
import pandas as pd 
import glob
from memory_profiler import memory_usage 

ModuleNotFoundError: No module named 'memory_profiler'

In [13]:
%load_ext rpy2.ipython %load_ext memory_profiler 

ModuleNotFoundError: No module named 'rpy2'

In [7]:
file_id = 26766812
x = requests.get(f'https://api.figshare.com/v2/file/download/{file_id}')

In [16]:
open('data.zip', 'wb').write(x.content)

814041183

In [19]:
with zipfile.ZipFile('data.zip', 'r') as zip_ref:
    zip_ref.extractall('data_extracted')

In [4]:
csvfiles = []
for file in glob.glob("data_extracted/*.csv"):
    csvfiles.append(file)

In [5]:
csvfiles

['data_extracted/MPI-ESM1-2-HR_daily_rainfall_NSW.csv',
 'data_extracted/ACCESS-CM2_daily_rainfall_NSW.csv',
 'data_extracted/SAM0-UNICON_daily_rainfall_NSW.csv',
 'data_extracted/EC-Earth3-Veg-LR_daily_rainfall_NSW.csv',
 'data_extracted/NorESM2-MM_daily_rainfall_NSW.csv',
 'data_extracted/NESM3_daily_rainfall_NSW.csv',
 'data_extracted/BCC-CSM2-MR_daily_rainfall_NSW.csv',
 'data_extracted/MRI-ESM2-0_daily_rainfall_NSW.csv',
 'data_extracted/NorESM2-LM_daily_rainfall_NSW.csv',
 'data_extracted/GFDL-ESM4_daily_rainfall_NSW.csv',
 'data_extracted/KIOST-ESM_daily_rainfall_NSW.csv',
 'data_extracted/GFDL-CM4_daily_rainfall_NSW.csv',
 'data_extracted/CMCC-ESM2_daily_rainfall_NSW.csv',
 'data_extracted/MPI-ESM-1-2-HAM_daily_rainfall_NSW.csv',
 'data_extracted/CMCC-CM2-SR5_daily_rainfall_NSW.csv',
 'data_extracted/FGOALS-f3-L_daily_rainfall_NSW.csv',
 'data_extracted/CMCC-CM2-HR4_daily_rainfall_NSW.csv',
 'data_extracted/INM-CM4-8_daily_rainfall_NSW.csv',
 'data_extracted/observed_daily_rain

In [12]:
%%time
%memit
big_df = []
count = 0
for csvfile in csvfiles:
    count +=1
    df = pd.read_csv(csvfile, index_col=None, header=0)
    df['model'] = csvfile.split("/")[1].split("_")[0] 
    big_df.append(df)
    if count==3:
        break
    
frame = pd.concat(big_df, axis=0, ignore_index=True)

UsageError: Line magic function `%memit` not found.


In [11]:
frame

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-36.467390,-35.532329,141.09375,142.03125,1.390174e-13,MPI-ESM1-2-HR
1,1889-01-02 12:00:00,-36.467390,-35.532329,141.09375,142.03125,1.335883e-13,MPI-ESM1-2-HR
2,1889-01-03 12:00:00,-36.467390,-35.532329,141.09375,142.03125,1.345589e-13,MPI-ESM1-2-HR
3,1889-01-04 12:00:00,-36.467390,-35.532329,141.09375,142.03125,1.252266e-13,MPI-ESM1-2-HR
4,1889-01-05 12:00:00,-36.467390,-35.532329,141.09375,142.03125,1.253846e-13,MPI-ESM1-2-HR
...,...,...,...,...,...,...,...
10628228,2014-12-27 12:00:00,-30.157068,-29.214660,153.12500,154.37500,6.689683e+00,SAM0-UNICON
10628229,2014-12-28 12:00:00,-30.157068,-29.214660,153.12500,154.37500,7.862555e+00,SAM0-UNICON
10628230,2014-12-29 12:00:00,-30.157068,-29.214660,153.12500,154.37500,1.000503e+01,SAM0-UNICON
10628231,2014-12-30 12:00:00,-30.157068,-29.214660,153.12500,154.37500,8.541592e+00,SAM0-UNICON
